In [ ]:
pip install ratelimiter

In [ ]:
import pandas as pd
import numpy as np
import requests
from time import sleep
import datetime
from ratelimiter import RateLimiter
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#creation de la liste earoports europe dans lesquelles lufthansa opère
airports = pd.read_csv("airports.csv", sep = ";")
airports["Continent"] = airports["TimeZoneId"].apply(lambda x : x.split("/")[0])
europe = airports[airports['Continent']== "Europe"].reset_index(drop=True)["AirportCode"]

In [ ]:
#creation des listes individuels 
np.random.seed(123)

ousmane , abdoul , abdel = np.array_split(np.random.permutation(europe), 3)

In [ ]:
#collecte de flight_information
def get_flight_information(origins, date, access_tokens):
    flights = []

    rate_limiter = RateLimiter(max_calls=4, period=1)  # 4 requêtes par seconde
    api_key_index = 0
    requests_count = 0

    for origin in origins:
        for hour in range(24):
            for minute in ["00", "30"]:
                with rate_limiter:
                    if requests_count >= 750:
                        print(f"Changement de clé API : utilisation de la clé {api_key_index + 1}") 
                        api_key_index += 1
                        requests_count = 0
                        if api_key_index >= len(access_tokens):
                            print("Toutes les clés API ont atteint la limite de requêtes.")
                            return pd.DataFrame.from_dict(flights).drop_duplicates().reset_index(drop=True)

                    time_str = f"{hour:02d}:{minute}"
                    datetime = f"{date}T{time_str}"
                    
                    authentification = {"Authorization": "Bearer " + access_tokens[api_key_index]}

                    try:
                        response = requests.get(
                            ("https://api.lufthansa.com/v1/operations/customerflightinformation/departures/{origin}/{datetime}"
                             "?limit=100&offset=0"
                            ).format(origin=origin, datetime=datetime),
                            headers=authentification).json()

                        for flight in response['FlightInformation']['Flights']['Flight']:
                            # Extraction des données du vol 
                            departure = flight.get('Departure', {})
                            arrival = flight.get('Arrival',{})
                            operatingCarrier = flight.get('OperatingCarrier')
                            equipment = flight.get('Equipment')
                            status = flight.get('Status')


                            row = {
                                'DpAirportCode': departure.get('AirportCode'),
                                'DpScheduledDate': departure.get('Scheduled', {}).get('Date'),
                                'DpScheduledTime': departure.get('Scheduled', {}).get('Time'),
                                'DpActualDate': departure.get('Actual', {}).get('Date'),
                                'DpActualTime': departure.get('Actual', {}).get('Time'),
                                'DpTerminalName': departure.get('Terminal', {}).get('Name'),
                                'DpTerminalGate': departure.get('Terminal', {}).get('Gate'),
                                'DpStatusCode': departure.get('Status', {}).get('Code'),
                                'DPStatusDescription': departure.get('Status', {}).get('Description'),
                                'ArrAirportCode':arrival.get('AirportCode', {}),
                                'ArrScheduledDate': arrival.get('Scheduled', {}).get('Date'),
                                'ArrpScheduledTime': arrival.get('Scheduled', {}).get('Time'),
                                'ArrActualDate':arrival.get('Actual', {}).get('Date'),
                                'ArrActualTime':arrival.get('Actual', {}).get('Time'),
                                'ArrTerminalName':arrival.get('Terminal', {}).get('Name'),
                                'ArrTerminalGate':arrival.get('Terminal', {}).get('Gate'),
                                'ArrStatusCode': arrival.get('Status', {}).get('Code'),
                                'ArrStatusDescription': arrival.get('Status', {}).get('Description'),                             
                                'AirlineID': operatingCarrier.get('AirlineID'),
                                'FlightNumber': operatingCarrier.get('FlightNumber'),
                                'AircraftCode': equipment.get('AircraftCode'),
                                'StatusCode' : status.get('Code'),
                                'StatusDescription' : status.get('Description')

                            }

                            flights.append(row)

                        requests_count += 1

                    except Exception as e:
                        print(f"Error for origin {origin}, time {time_str}: {e}")

    flight_information_departures = pd.DataFrame.from_dict(flights).reset_index(drop = True)
    flight_information_departures = flight_information_departures.drop_duplicates().reset_index(drop = True)
    flight_information_departures.insert(0, 'FlightId', flight_information_departures['AirlineID'] + flight_information_departures['FlightNumber'].astype(str))
    return flight_information_departures

access_tokens = ["dgwt2cn2z9d39qwae7ehxmen", "apj5gpxj9c9ut9up6g68gsxs",
                 "5a5zsvd83ym6nb4mz47ddjds", "s7j65m93x5ap6ne7qfk4dkva",
                 "4hv9m2n4epq83z7j3g5j8ne8", "mucjtvh5vnbvuhpsktepesns",
                 "ztwk3etpxtvbnzmq5ex7vq8t", "fvzvx8ksbyrvgz4xnyzd8qtk",]

origins = ["LHR","CDG","AMS","FRA","MAD","BCN","MUC","LGW","FCO","DUB","ORY"]
date = "2023-05-17"

flight_information_departures = get_flight_information_departures(origins, date, access_tokens)
flight_information_departures.to_csv("/content/drive/MyDrive/flight_information.csv", sep = ";", index=False)
display(flight_information_departures)
